# Data Preprocessing
Studenten: Wietske Boersma, Isa van den Hoven, Alecsandru Kreefft-Libiu, Timber Rijsdorp

Teamnummer: N5

In [ ]:
import pandas as pd

## Dataset 1: Kanker

Deze dataset bevat informatie over de hoeveelheid mensen die overlijden aan verschillende soorten kanker. De data laat per kankersoort zien wat de leeftijd gestandaardiseerde hoeveelheid mensen zijn die overleden zijn aan die kankersoort in dat jaar en in dat land. Wij zijn voor ons onderzoek alleen geinteresseerd in de data rondom longkanker omdat die kankersoort voornamelijk wordt benoemd in discussie rondom luchtvervuiling. 

Hieronder kan je zien hoe de data word't gefilterd. We selecteren alleen de kolommen: landcode, jaar en longkanker overledenen, en we verwijderen datapunten dat niet verbonden is aan een land.

In [ ]:
# https://www.kaggle.com/datasets/programmerrdai/cancer?select=cancer-death-rates-by-type.csv
# Load data
df_cancer_rate = pd.read_csv('datasets/cancer-death-rates-by-type.csv')
# Select columns
df_cancer_rate = df_cancer_rate[['Code', 'Year', 'Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: Age-standardized (Rate)']]
# Rename columns
df_cancer_rate = df_cancer_rate.rename(columns={'Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: Age-standardized (Rate)':'TBL rate'})
# Delete non-country entries
df_cancer_rate = df_cancer_rate[df_cancer_rate['Code'].notna()]
# Display dataframe
df_cancer_rate.head(5)

## Dataset 2: Luchtvervuiling

In deze dataset staat de hoeveelheid vervuiling voor verschillende stoffen dat is gemeten per land per jaar. We zijn geinteresseerd in alle soorten uitstoot voor dit onderzoek, dus de enige variabel dat we eruit filteren is het land omdat de landcode deze informatie al geeft. Verder verwijderen we weer datapunten dat niet verbonden is met een land.

In [ ]:
# https://ourworldindata.org/explorers/air-pollution?tab=table&uniformYAxis=0&Pollutant=All+pollutants&Sector=From+all+sectors+%28Total%29&Per+capita=false&country=USA~CHN~IND~GBR~OWID_WRL
# Load data
df_pollution = pd.read_csv('datasets/air-pollution.csv')
# Select columns
df_pollution = df_pollution.drop('Entity', axis=1)
# Delete non-country entries
df_pollution = df_pollution[df_pollution['Code'].notna()]
# Display dataframe
df_pollution.head(5)

## Dataset 3: Roken

Voor de laatste dataset hebben we gekeken naar het percentage van de mensen per land dat rookt. Deze dataset bevat ook per land per jaar het percentage dagelijkse rokers over de gehele populatie. We selecteren deze kolommen en verwijderen de niet-landelijke data. 

In [ ]:
# https://ourworldindata.org/grapher/daily-smoking-prevalence-bounds?tab=chart
# Load data
df_smoking = pd.read_csv('datasets/smoking.csv')
# Select columns
df_smoking = df_smoking[['Code', 'Year', 'Daily smoking prevalence - both (IHME, GHDx (2012))']]
# Rename columns
df_smoking = df_smoking.rename(columns={'Daily smoking prevalence - both (IHME, GHDx (2012))':'Percentage daily smokers'})
# Delete non-country entries
df_smoking = df_smoking[df_smoking['Code'].notna()]
# Display dataframe
df_smoking.head(5)

## Dataset 4: World statistieken

We gebruiken deze dataset om de uitstoot gegevens naar een getal te berekenen die makkelijker te visualiseren zijn. De dataset bevat allerlei statistieken van elk land, maar wij gebruiken alleen de populatie variabel.

In [ ]:
# https://www.kaggle.com/datasets/mutindafestus/world-statistics-dataset-from-world-bank
# Load data
df_countries = pd.read_csv('datasets/countries.csv')
# Select columns
df_countries = df_countries[['Country Code', 'year', 'Population, total']]
# Rename columns
df_countries = df_countries.rename(columns={'Country Code': 'Code', 'year':'Year', 'Population, total':'Population'})
# Delete non-country entries
df_countries = df_countries[df_countries['Code'].notna()]
# Display dataframe
df_countries.head(5)

## Filtreren

Alle datasets bevatten jaarlijke data, maar niet alle datasets hebben dezelfde jaartallen. Om te zorgen dat we geen lege data krijgen in jaren dat een dataset niet heeft, filtreren we alle datasets naar alle jaren tussen 1990 en 2012. 

In [ ]:
# Select years to collect data from
begin_year = 1990
end_year = 2012
df_cancer_year = df_cancer_rate[df_cancer_rate['Year'].between(begin_year, end_year)]
df_smoking_year = df_smoking[df_smoking['Year'].between(begin_year, end_year)]
df_pollution_year = df_pollution[df_pollution['Year'].between(begin_year, end_year)]
df_countries_year = df_countries[df_countries['Year'].between(begin_year, end_year)]

## Combineren
Nu gaan we de datasets samenvoegen. Dit doen we door de datasets te mergen op de landcode en jaar. Verder wordt nog de overige data dat uit landen komt die niet voorkomen in de rest van de datasets verwijdert.

In [ ]:
# Merge dataframes on country code and year
df_combined = pd.merge(left=df_smoking_year, right=df_cancer_year, how='left', on=['Code', 'Year'])
df_combined = pd.merge(left=df_pollution_year, right=df_combined, how='left', on=['Code', 'Year'])
# Only select entries that are represented in all datasets
df_combined = df_combined[df_combined['TBL rate'].notna()]
df_combined = df_combined[df_combined['Percentage daily smokers'].notna()]

# Display data
df_combined.head(5)

## Per duizend inwoners

Voor sommige grafieken willen we in plaats van de totale uitstoot de uitstoot per capita hebben. Om deze date te krijgen delen we elke uitstoot door de totale populatie van het land waar het is gemeten. We vermenigvuldigen de uitkomst met 1000 om de getallen makkelijker te kunnen gebruiken.

In [ ]:
# Select columns to be divided by population
columns = ['Nitrogen oxide (NOx)', 'Sulphur dioxide (SO₂) emissions', 'Carbon monoxide (CO) emissions', 'Organic carbon (OC) emissions', 'Non-methane volatile organic compounds (NMVOC) emissions', 'Black carbon (BC) emissions', 'Ammonia (NH₃) emissions']
# Combine the population statistics to the dataframe
df_norm = pd.merge(left=df_countries_year, right=df_combined, how='left', on=['Code', 'Year'])
# Remove invalid data
df_norm = df_norm[df_norm['TBL rate'].notna()]
df_norm = df_norm[df_norm['Percentage daily smokers'].notna()]
# Divide by population and multiply it by 1000
for column in columns:
    df_norm[column] = (df_norm[column]/df_norm['Population']) * 1000
# Remove the population column
df_norm = df_norm.drop('Population', axis=1)
# Display data
df_norm.head(5)

## Opslaan

En als laatst wordt de gecombineerde dataset opgeslagen zodat het makkelijk toegangbaar is voor het onderzoek.

In [ ]:
# Save to csv
df_combined.to_csv('datasets/combined.csv')
df_norm.to_csv('datasets/norm.csv')